In [90]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
lemmatizer = nltk.stem.WordNetLemmatizer()
import random

In [91]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [92]:
data = pd.read_csv('Moneypay.txt', sep= ':', header = None)
data.head(30)

,0,1
0,Question,'How do I set up an MoneyPay merchant account'
1,Answer,'It's really easy! Go to Google Play and down...
2,Question,'What documents are required to complete my r...
3,Answer,'Your Bank Verification Number (BVN).A utilit...
4,Question,'Can I proceed with registration if I do not ...
5,Answer,"'Yes, you can proceed to register as an Mon..."
6,Question,'What do I do after registering as an MoneyPa...
7,Answer,'Start sending or receiving money! Remember...
8,Question,'Is there a limit on the maximum transaction ...
9,Answer,'You can receive and send up to 5 million N...


In [93]:
question = data.loc[data[0] == 'Question']
answer = data.loc[data[0] == 'Answer']

question = question.reset_index(drop = True)
answer = answer.reset_index(drop = True)

df = pd.DataFrame()
df['Customer'] =  question[1]
df['Response'] = answer[1]

df.head()

,Customer,Response
0,'How do I set up an MoneyPay merchant account','It's really easy! Go to Google Play and down...
1,'What documents are required to complete my r...,'Your Bank Verification Number (BVN).A utilit...
2,'Can I proceed with registration if I do not ...,"'Yes, you can proceed to register as an Mon..."
3,'What do I do after registering as an MoneyPa...,'Start sending or receiving money! Remember...
4,'Is there a limit on the maximum transaction ...,'You can receive and send up to 5 million N...


In [94]:
ds = df.copy()
ds.to_csv('cleaned_moneypay.txt', index=False)

In [96]:
def preprocess_text(text):
    sentences =nltk.sent_tokenize(text)

    preprocessed_sentences = []
    for sentence in sentences:
        tokens = [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(sentence) if word.isalnum()]
        preprocessed_sentence = ' '.join(tokens)
        preprocessed_sentences.append(preprocessed_sentence)
    
    return ' '.join(preprocessed_sentences)


df['tokenized Customer'] = df['Customer'].apply(preprocess_text)
df


,Customer,Response,tokenized Customer
0,'How do I set up an MoneyPay merchant account','It's really easy! Go to Google Play and down...,do i set up an moneypay merchant account
1,'What documents are required to complete my r...,'Your Bank Verification Number (BVN).A utilit...,document are required to complete my registration
2,'Can I proceed with registration if I do not ...,"'Yes, you can proceed to register as an Mon...",i proceed with registration if i do not have t...
3,'What do I do after registering as an MoneyPa...,'Start sending or receiving money! Remember...,do i do after registering a an moneypay merchant
4,'Is there a limit on the maximum transaction ...,'You can receive and send up to 5 million N...,there a limit on the maximum transaction amoun...
5,What do I do when a transaction is made via ...,Please report this immediately in the Supp...,what do i do when a transaction is made via th...
6,How long does it take for a reversal to happen?,Reversals on the system happen instantly t...,how long doe it take for a reversal to happen
7,'What do I do if my POS shows that the rever...,'Please ensure that the reversal process c...,do i do if my po show that the reversal is app...
8,What do I do when I receive a debit alert bu...,'Please contact ng-support@MoneyPay-inc.com a...,what do i do when i receive a debit alert but ...
9,'What do I do when I am debited more than w...,'Please send your bank statement with the...,do i do when i am debited more than what is in...


In [97]:
xtrain = df['tokenized Customer'].to_list()
xtrain

['do i set up an moneypay merchant account',
 'document are required to complete my registration',
 'i proceed with registration if i do not have the corporate affair commission cac or business name document',
 'do i do after registering a an moneypay merchant',
 'there a limit on the maximum transaction amount for a merchant',
 'what do i do when a transaction is made via the po but i do not see the fund credited in my wallet',
 'how long doe it take for a reversal to happen',
 'do i do if my po show that the reversal is approved but cardholder s account is still debited please request for the cardholder to resolve this with their bank',
 'what do i do when i receive a debit alert but there is no credit in my wallet',
 'do i do when i am debited more than what is in my wallet',
 'do i do when my wallet get debited twice for one bank transfer but the transfer is only shown once in the receiver account',
 'do i do when my wallet get debited and the person did not receive the money i sen

In [98]:
tfidf_vectorizer = TfidfVectorizer()
corpus = tfidf_vectorizer.fit_transform(xtrain)

print(corpus[0])

  (0, 0)	0.31970685132330934
  (0, 40)	0.35370099238928193
  (0, 43)	0.39752706262738885
  (0, 6)	0.39752706262738885
  (0, 84)	0.4592964393410821
  (0, 69)	0.4592964393410821
  (0, 23)	0.1863361687237884


In [99]:
def responder(test):
    user_input_processed = preprocess_text(test)
    vectorized_user_input = tfidf_vectorizer.transform([user_input_processed])
    similarity_score = cosine_similarity(vectorized_user_input, corpus)
    argument_maximum = similarity_score.argmax()

    print(df['Response'].iloc[argument_maximum])

bot_greetings = ['Hello user, Welcome to Moneypay, I am Missy, How can I help you?',
    'Welcome to MoneyPay, I am Missy, How are you today? How may I be of service to you?',
    'Missy here, This is MoneyPay, is there anything I can do for you?',
    'I am Missy and this is MoneyPay platform, is there anyway I can be of service to you?',
    'Welcome to our MoneyPay platform, My name is Missy, How can I help you?'
             ]

bot_farewell = ['Thanks for checking out MoneyPay, bye',
             'Hope, I have been helpful, bye',
             'Hope to see you soon',
             'Do not hesiatate to contact me again',
             'See ya..bye'
           ]

human_greetings = ['hi', 'hello there', 'hey','hello']
human_exits = ['thanks bye', 'bye', 'quit','bye bye', 'close', 'exit']


random_greeting = random.choice(bot_greetings)
random_farewell = random.choice(bot_farewell)

while True:
    user_input = input('You: ')

    if user_input.lower() in human_greetings:
        print('You                                  ' + user_input)
        print(random_greeting)
    elif user_input.lower() in human_exits:
        print(random_farewell)
        break
    else:
        responder(user_input)



Thanks for checking out MoneyPay, bye
